In [1]:
import pandas as pd 
import numpy as np 
import re
import json
import os 
from sentence_transformers import SentenceTransformer
import pickle

/Users/adithisatish/TUM/Academics/SoSe24/guided_research/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [53]:
airbnb_dir = "../../europeancities-wikivoyage-tripadvisor/data-sources/airbnb/cleaned/"
os.listdir(airbnb_dir)

['vienna-2024-04-19_cleaned.csv.gz',
 'berlin-2024-03-24_cleaned.csv.gz',
 'brussels-2024-04-25_cleaned.csv.gz',
 'amsterdam-2024-04-12_cleaned.csv.gz',
 'bordeaux-2024-04-22_cleaned.csv.gz',
 'valencia-2024-03-24_cleaned.csv.gz',
 'lyon-2024-04-22_cleaned.csv.gz',
 'rome-2024-04-19_cleaned.csv.gz',
 'prague-2024-03-25_cleaned.csv.gz',
 'porto-2024-04-22_cleaned.csv.gz',
 'bologna-2024-04-25_cleaned.csv.gz',
 'london-2024-04-17_cleaned.csv.gz',
 'barcelona-2024-04-17_cleaned.csv.gz',
 'stockholm-2024-03-30_cleaned.csv.gz',
 'geneva-2023-12-27_cleaned.csv.gz',
 'munich-2024-04-28_cleaned.csv.gz',
 'naples-2024-04-25_cleaned.csv.gz',
 'madrid-2024-04-19_cleaned.csv.gz',
 'thessaloniki-2023-12-25_cleaned.csv.gz',
 'copenhagen-2024-03-30_cleaned.csv.gz',
 'istanbul-2024-03-31_cleaned.csv.gz',
 'paris-2024-04-17_cleaned.csv.gz',
 'zurich-2024-03-30_cleaned.csv.gz',
 'milan-2024-03-24_cleaned.csv.gz',
 'riga-2024-03-31_cleaned.csv.gz']

In [54]:
df = pd.read_csv(airbnb_dir + 'zurich-2024-03-30_cleaned.csv.gz')
df.columns

Index(['Unnamed: 0', 'listing_url', 'name', 'description', 'host_name',
       'property_type', 'room_type', 'accommodates', 'price', 'minimum_nights',
       'maximum_nights'],
      dtype='object')

In [67]:
df['room_type'].unique()

array(['Private room', 'Entire home/apt', 'Shared room'], dtype=object)

In [80]:
def remove_html(s):
    return re.sub("<[^>]*>", "", s)

def preprocess_csv(df):
    df = df[['listing_url', 'name', 'description', 'host_name', 'room_type', 'accommodates', 'price']]
    airbnb = df
    airbnb['description'] = airbnb['description'].apply(remove_html)
    combined = []
    for i, row in airbnb.iterrows():
        combined.append(f"name: {row['name']}, description: {row['description']}, accommodates: {row['accommodates']}, price: {row['price']}")

    airbnb['combined'] = combined
    # airbnb.head()

    return airbnb

In [81]:
airbnb_embeddings = "../../europeancities-wikivoyage-tripadvisor/data-sources/airbnb/embeddings/"

In [82]:
# airbnb_combined = pd.DataFrame()
# for file in os.listdir(airbnb_dir):
#     df = pd.read_csv(airbnb_dir + file)
    # print(file, len(preprocess_csv(df)))
    # airbnb_combined = pd.concat([airbnb_combined, preprocess_csv(df)])

# len(airbnb_combined)

# embeddings = []

def compute_airbnb_embeddings():
    wikivoyage_embeddings = []
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    vector_dimension = model.get_sentence_embedding_dimension()

    for file in os.listdir(airbnb_dir):
        embeddings = []

        city = file.split("-")[0]
        df = pd.read_csv(airbnb_dir + file)
        df = preprocess_csv(df)
        
        print(f"Computing embeddings for {city.capitalize()}")
        for i, row in df.iterrows():
            emb = model.encode(row['combined']).tolist()
            embeddings.append(emb)
        
        print(f"Computed embeddings, now saving file for {city.capitalize()}")
        df['embeddings'] = embeddings
        df.to_csv(airbnb_embeddings + city + "_embeddings.csv")

In [83]:
compute_airbnb_embeddings()

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_93319/2633167890.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb['description'] = airbnb['description'].apply(remove_html)
/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_93319/2633167890.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb['combined'] = combined


Computing embeddings for Vienna


KeyboardInterrupt: 

In [15]:
# from qdrant_client import QdrantClient

# client = QdrantClient(
#     url="https://cb8144d3-3bc4-4b48-9b8f-0d5162835b62.us-east4-0.gcp.cloud.qdrant.io:6333", 
#     api_key="Kxe4k8QDskHicKFj7DVMkLSHW1A7jApqX1nCN09jm1WPH9ycBVFO8Q",
# )

In [ ]:
# from qdrant_client.models import VectorParams, Distance

# client.delete_collection(collection_name="airbnb")

# client.recreate_collection(
#    collection_name="airbnb",
#    vectors_config=VectorParams(size=vector_dimension, distance=Distance.COSINE, on_disk=True),
# )